In [ ]:
!pip install google-search-results

In [1]:
from serpapi import GoogleSearch
from urllib.parse import *

**Mencari restoran dengan review > 150**

In [ ]:
params = {
  "engine": "google_maps",
  "q": "restoran",
  "ll": "@-7.2756141,112.6426429,12z",
  "type": "search",
  "api_key": "d622018760727c22f5b1ffc9944319d2c91142b402fb9443371158deba13b312"
}

search = GoogleSearch(params)

mPosition = 0 # Angka indeks dari data
mNums = 5 # Batas jumlah data yang ditampilkan
local_results = []

while mPosition <= mNums:
  results = search.get_dict()

  for result in results["local_results"]:
    mPosition += 1
    
    if mPosition <= mNums:
      
      # Jika jumlah review kurang dari 150, maka jangan masukkan datanya
      if result["reviews"] < 150:
        mNums += 1
        continue
        
      # Append data sesuai dengan batas jumlah yang sudah ditentukan
      local_results.append({'title': result["title"],
                           'data_id': result["data_id"],
                           'reviews_total': result["reviews"]});

  if "next" in results.get("serpapi_pagination", {}):
    # Akan mengubah parameter dari 'GoogleSearch()' dengan isi parameter dari halaman selanjutnya
    search.params_dict.update(dict(parse_qsl(urlsplit(results.get("serpapi_pagination").get("next")).query)))

  else:
    break

**Mencari review berdasarkan restoran**

In [ ]:
data = {'restaurant_name':[],
        'name':[],
        'rating':[],
        'review':[]}

for lr in local_results:
  
  params = {
    "engine": "google_maps_reviews",
    "data_id": "",
    "api_key": "d622018760727c22f5b1ffc9944319d2c91142b402fb9443371158deba13b312"
  }
  
  params["data_id"] = lr['data_id']

  search = GoogleSearch(params)

  mPosition = 0 # Angka indeks dari data
  mNums = 50 # Batas jumlah data yang ditampilkan

  while mPosition <= mNums:
    results = search.get_dict()

    for result in results["reviews"]:
      mPosition += 1

      if mPosition <= mNums:
        # Append data sesuai dengan batas jumlah yang sudah ditentukan
        data['restaurant_name'].append(lr['title']);
        data['name'].append(result["user"]["name"]);
        data['rating'].append(result["rating"]);
        data['review'].append(result["snippet"]);

    if "next" in results.get("serpapi_pagination", {}):
      # Akan mengubah parameter dari 'GoogleSearch()' dengan isi parameter dari halaman selanjutnya
      search.params_dict.update(dict(parse_qsl(urlsplit(results.get("serpapi_pagination").get("next")).query)))

    else:
      break
    
review_original = data['review']

**Ekspor ke CSV sebelum preprocessing**

In [16]:
import pandas as pd
df_original = pd.DataFrame(data)
df_original.to_csv('review.csv', index=False)

In [15]:
df_original

,restaurant_name,name,rating,review
0,Merr Kayu Manis Restaurant,Erik Widiyanto (AKIM),5.0,"This is small foodcourt, they have several pop..."
1,Merr Kayu Manis Restaurant,Frieddy Kurniawan,5.0,This restaurant provide many kind of famous fo...
2,Merr Kayu Manis Restaurant,Alvin Jusuf,4.0,"The place not bad, also the foods pretty good ..."
3,Merr Kayu Manis Restaurant,Yoseph J,5.0,"Price is really cheap, food was great. Ambienc..."
4,Merr Kayu Manis Restaurant,Bill,3.0,"The food was normal, some are good some are ba..."
...,...,...,...,...
245,Restoran Sederhana Masakan Padang,dudi Purwa,5.0,(Translated by Google) all the dishes are deli...
246,Restoran Sederhana Masakan Padang,Daniel Nugroho,1.0,(Translated by Google) Not good. It's differen...
247,Restoran Sederhana Masakan Padang,chelsea shelly,3.0,(Translated by Google) The cooking is deliciou...
248,Restoran Sederhana Masakan Padang,Heru Tridarto Tirtowardojo,4.0,(Translated by Google) Located on Jl. Dharmawa...


# **Preprocessing**

**Case Folding**

In [17]:
import re
import string

In [18]:
review_remove_translated = []

for review in review_original:
  review_sep = review.split("(Translated by Google) ")
  
  # Jika terdapat (Translated by Google)
  if review_sep[0] == "":
    review_sep = ("".join(review_sep)).split("(Original)")
    review_sep = review_sep[0]
    review = "".join(review_sep)
    
  review_remove_translated.append(review)

In [19]:
review_clean = []

for review in review_remove_translated:
    # Remove Unicode
    review_test = re.sub(r'[^\x00-\x7F]+', ' ', review)
    # Remove Mentions
    review_test = re.sub(r'@\w+', '', review_test)
    # Lowercase the document
    review_test = review_test.lower()
    # Remove punctuations
    review_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', review_test)
    # Remove the numbers
    review_test = re.sub(r'[0-9]', '', review_test)
    # Remove the doubled space
    review_test = re.sub(r'\s{2,}', ' ', review_test)
    
    review_clean.append(review_test)

**Tokenizing**

In [23]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [24]:
review_tokenized = []

for review in review_clean:
  review_tokenized.append(word_tokenize(review))

**Filtering (StopWord Removal)**

In [25]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [26]:
review_filtered = []
listStopword =  set(stopwords.words('english'))

for review in review_tokenized:
  
  review_each_filtered = []
  
  for r in review:
    if r not in listStopword:
        review_each_filtered.append(r)
        
  review_filtered.append(review_each_filtered)

**Stemming (LancasterStemmer Algorithm)**

In [28]:
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer() 

review_stemmed = []
    
for review in review_filtered:
  
  review_each_stemmed = []
  
  for r in review:
    review_each_stemmed.append(stemmer.stem(r))
        
  review_stemmed.append(review_each_stemmed)

**Ekspor ke CSV hasil preprocessing**

In [32]:
data_preprocessed = data
data_preprocessed['review'] = review_stemmed
df = pd.DataFrame(data_preprocessed)
df.to_csv('review_processed.csv', index=False)